In [54]:
import pandas as pd
import glob, os, re, json
from tqdm.notebook import tqdm
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [6]:
files = glob.glob("../raw/[0-9]*.csv")

In [7]:
dfs = []
for fi in tqdm(files):
    try:
        df = pd.read_csv(fi, index_col=0)
    except Exception:
        df = pd.read_csv(fi, encoding="windows-1252", index_col=0)
    dfs.append(df)
full_data = pd.concat(dfs, axis = 0)

  0%|          | 0/701 [00:00<?, ?it/s]

In [9]:
full_text = full_data.dropna(subset=["Answer"])
full_text.head()

,LineNo,Interviewer,Statement,Question,Answer,Vol,Page,Area,Witness,Location
0,1291,The Chairman.,NaN,What is your occupation ?,"My son is a crofter, and I am in my son's plac...",1.0,73.0,"Skye, Skeabost",Hugh Mcnab,"Kildonan, Lynedale"
1,1292,The Chairman.,NaN,How far back with the previous landlord?,Thirty-four years ago.,1.0,73.0,"Skye, Skeabost",Hugh Mcnab,"Kildonan, Lynedale"
2,1293,Mr Fraser-Mackintosh.,NaN,We will take the present landlord?,The present landlord raised our rent 31s. and ...,1.0,73.0,"Skye, Skeabost",Hugh Mcnab,"Kildonan, Lynedale"
3,1294,The Chairman.,NaN,Will you state the amount of stock which your ...,In summer and autumn we could keep two cows an...,1.0,73.0,"Skye, Skeabost",Hugh Mcnab,"Kildonan, Lynedale"
4,1295,Sheriff Nicolson.,NaN,When were the sheep taken from you?,The landlord took the sheep stock from us a fe...,1.0,73.0,"Skye, Skeabost",Hugh Mcnab,"Kildonan, Lynedale"


In [26]:
df = pd.read_csv("../data/output/sent_meta.csv")
df.columns = ["text", "line_num", "Area", "Witness", "Location"]
df.loc[df['text'].isna(),'text'] = ""
df.head()

,text,line_num,Area,Witness,Location
0,You have a statement to make,1,"Edinburgh,ï¿½",Alexander Robertsonï¿½,Edinburgh
1,The testimony I propose to adduce is founded o...,1,"Edinburgh,ï¿½",Alexander Robertsonï¿½,Edinburgh
2,I have also during the last forty years sought...,1,"Edinburgh,ï¿½",Alexander Robertsonï¿½,Edinburgh
3,The result of my inquiries was to satisfy me t...,1,"Edinburgh,ï¿½",Alexander Robertsonï¿½,Edinburgh
4,As your time is short I shall pass over a vari...,1,"Edinburgh,ï¿½",Alexander Robertsonï¿½,Edinburgh


In [28]:
analyzer = SentimentIntensityAnalyzer()
tqdm.pandas(desc="sentiment")
df['sentiment'] = df['text'].progress_apply(lambda x: analyzer.polarity_scores(x)["compound"])

sentiment:   0%|          | 0/109151 [00:00<?, ?it/s]

In [29]:
df.to_csv("sent_sentiments.csv", index = False)